### `Task` How dimensionality reduction using Principal Component Analysis (PCA) on the Wine Quality dataset contributes to improving the classification accuracy and efficiency of wine type.

Note : Use KNN for Classification.

Data Link :  [Wine Data](https://docs.google.com/spreadsheets/d/e/2PACX-1vQDVwxneOKOaJL13QMhkAhYrgWlH1tICY7RacUnj_lL8m9uUWaaUf3p7bScNyh_D2Rvt7nc1q11adSy/pub?gid=647503637&single=true&output=csv)

In [2]:
# Data Loading
import pandas as pd
wine_data_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQDVwxneOKOaJL13QMhkAhYrgWlH1tICY7RacUnj_lL8m9uUWaaUf3p7bScNyh_D2Rvt7nc1q11adSy/pub?gid=647503637&single=true&output=csv"
wine = pd.read_csv(wine_data_path)
wine.head(1)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.001,3.0,0.45,8.8,6


In [3]:
wine.shape

(6497, 13)

In [11]:
wine.isnull().sum()

,0
type,0
fixed acidity,10
volatile acidity,8
citric acid,3
residual sugar,2
chlorides,2
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,9


In [12]:
from sklearn.impute import KNNImputer
knn = KNNImputer(n_neighbors=3)

In [15]:
wine_trs = knn.fit_transform(wine.drop(['type'],axis = 1))

In [22]:
temp = wine.drop(['type'],axis=1).columns

In [23]:
df = pd.DataFrame(wine_trs,columns=temp)

In [24]:
df['type'] = wine['type']

In [26]:
df.isnull().sum()

,0
fixed acidity,0
volatile acidity,0
citric acid,0
residual sugar,0
chlorides,0
free sulfur dioxide,0
total sulfur dioxide,0
density,0
pH,0
sulphates,0


In [27]:
### At first lets out how many column have to be placed

In [53]:
from sklearn.decomposition import PCA

In [54]:
pca = PCA(n_components=None) #We want to make pca equal to the total dimension i.e. 13

In [55]:
from sklearn.model_selection import train_test_split

In [69]:
column = df.iloc[:,-1:]
X = df.drop(column,axis=1)
Y = df.iloc[:,-1:]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=43)

In [70]:
X_train.shape,X_test.shape

((5197, 12), (1300, 12))

In [71]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [72]:
X_train.shape

(5197, 12)

In [73]:
X_train_trs = pca.fit_transform(X_train)
X_test_trs = pca.transform(X_test)

In [74]:
X_train_trs.shape

(5197, 12)

In [75]:
pca.components_.shape

(12, 12)

In [76]:
## These means there are 12 PCA in 12 dimension

In [77]:
pca.explained_variance_ratio_

array([0.25393333, 0.22321747, 0.13661278, 0.08978521, 0.0684627 ,
       0.05594973, 0.04620419, 0.04221126, 0.03723327, 0.0249623 ,
       0.01888382, 0.00254394])

In [83]:
import numpy as np
communative_variance = np.cumsum(pca.explained_variance_ratio_)
communative_variance

array([0.25393333, 0.4771508 , 0.61376358, 0.70354879, 0.77201149,
       0.82796122, 0.87416541, 0.91637667, 0.95360994, 0.97857224,
       0.99745606, 1.        ])

In [82]:
np.argmax(communative_variance>=0.9)

7

In [97]:
#It means that we can take the minimum of 0-7 i.e 8 component here to achieve 90 variance.This is done to avoid blind selection of no of components which
# can lead to decreased result variance.Now without any problem,we can take minimum i.e. 8 components

In [84]:
pc = PCA(n_components=8)

In [85]:
X_trn = pc.fit_transform(X_train)
X_tst = pc.transform(X_test)

In [86]:
X_trn.shape

(5197, 8)

In [87]:
from sklearn.neighbors import KNeighborsClassifier
knc = KNeighborsClassifier()

In [91]:
knc.fit(X_trn,Y_train)
# Y_train.shape

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [93]:
Y_pred = knc.predict(X_tst)

In [94]:
from sklearn.metrics import accuracy_score
accu = accuracy_score(Y_test,Y_pred)

In [95]:
accu

0.9907692307692307

In [96]:
### Here accuracy is of 99% with 8 Principle Components instead of 13